In [2]:
import pandas as pd
import os # use this to access your environment variables
import requests # this will be |used to call the APIs
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
from scipy import stats
import json #json parsing libraries

##### 1.Build a regression model using Python’s statsmodels module that demonstrates a relationship between the number of bikes in a particular location and the characteristics of the POIs in that location.
##### 2.Interpret results. Expand on the model output, and derive insights from your model.
##### 3.Stretch: can you think of a way to turn the above regression problem into a classification one? Without coding, can you sketch out how you would cast the problem specifically, and lay out your approaches?

In [3]:
df = pd.read_json(r'Chi_9am_mon_fulldata.json')

In [4]:
#now let's create a dataframe just holding the information we care about
df2 = df[['slots','avail_bikes_9am','change_8_to_830','change_830_to_9','change_8_to_9','distance','num_stations_1mi','num_stations_2mi','all_stations','transit_cat']]

In [5]:
df.head()

,comp_id,name,latitude,longitude,slots,avail_bikes_9am,renting,timestamp,distance,near_station,num_stations_1mi,num_stations_2mi,all_stations,transit_cat,change_8_to_830,change_830_to_9,change_8_to_9
0,divvy,Lake Park Ave & 56th St,41.793242,-87.587782,19,13,1,2023-11-20 15:01:15.831,653,"5311 S Lake Park Ave (at 53rd St), Chicago, IL...",8,2,10,3,-1,-1,-2
1,divvy,Ada St & Washington Blvd,41.882830,-87.661206,15,10,1,2023-11-20 15:01:13.757,384,"1200 W Randolph St, Chicago, IL 60607",9,1,10,3,-1,-1,-2
2,divvy,Ashland Ave & Grace St,41.950687,-87.668700,15,13,1,2023-11-20 15:01:16.001,432,1612 W Irving Park Rd (btwn Ashland Ave. & Pau...,10,0,10,3,0,-1,-1
3,divvy,Clark St & Wrightwood Ave,41.929546,-87.643118,15,7,1,2023-11-20 15:01:13.758,500,"2548 N Halsted St (at Wrightwood Ave), Chicago...",10,0,10,3,-2,-2,-4
4,divvy,Adler Planetarium,41.866095,-87.607267,39,12,1,2023-11-20 15:01:14.360,1421,"1416 S Michigan Ave, Chicago, IL 60605",5,5,10,3,0,0,0


In [6]:
df2.describe()

,slots,avail_bikes_9am,change_8_to_830,change_830_to_9,change_8_to_9,distance,num_stations_1mi,num_stations_2mi,all_stations,transit_cat
count,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000
mean,9.049910,3.600120,0.009020,-0.003007,0.006013,1577.116657,4.146723,5.364402,9.511124,2.821407
std,8.543893,4.783966,0.671253,0.809460,1.132588,1034.226487,3.111660,2.915700,1.001066,0.537386
min,1.000000,0.000000,-4.000000,-10.000000,-10.000000,3.000000,0.000000,0.000000,2.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,709.000000,2.000000,3.000000,9.000000,3.000000
50%,9.000000,1.000000,0.000000,0.000000,0.000000,1365.000000,3.000000,6.000000,10.000000,3.000000
75%,15.000000,6.000000,0.000000,0.000000,0.000000,2433.500000,6.000000,8.000000,10.000000,3.000000
max,55.000000,34.000000,9.000000,8.000000,13.000000,3515.000000,10.000000,10.000000,10.000000,3.000000


In [7]:
y = df2['change_8_to_9']
df_new = df2.drop(columns=['change_8_to_9'])

print(df_new.columns)

X = [sm.add_constant(df_new[column]) for column in df_new.columns]
X[1]

Index(['slots', 'avail_bikes_9am', 'change_8_to_830', 'change_830_to_9',
       'distance', 'num_stations_1mi', 'num_stations_2mi', 'all_stations',
       'transit_cat'],
      dtype='object')


,const,avail_bikes_9am
0,1.0,13
1,1.0,10
2,1.0,13
3,1.0,7
4,1.0,12
...,...,...
1658,1.0,3
1659,1.0,0
1660,1.0,7
1661,1.0,3


In [8]:
X[2]

,const,change_8_to_830
0,1.0,-1
1,1.0,-1
2,1.0,0
3,1.0,-2
4,1.0,0
...,...,...
1658,1.0,0
1659,1.0,0
1660,1.0,0
1661,1.0,0


In [9]:
Models = [sm.OLS(y,x) for x in X] #list of models
Results = [model.fit() for model in Models] #list of results
Adj_Rsquared = [results.rsquared_adj for results in Results] #list of rsquared
Pval = [results.pvalues for results in Results] #list of p-values

In [10]:
for i in range(len(Adj_Rsquared)):
     print(f'adj_R2: {Adj_Rsquared[i]:.3f}, P-values: {*Pval[i],}, column: {df2.columns[i]}')

adj_R2: 0.013, P-values: (0.0007898493290166014, 1.4956518539117485e-06), column: slots
adj_R2: 0.041, P-values: (8.747339303813141e-07, 4.0847238829841874e-17), column: avail_bikes_9am
adj_R2: 0.502, P-values: (0.8073707956824908, 2.8031446609050668e-254), column: change_8_to_830
adj_R2: 0.658, P-values: (0.5618556775234632, 0.0), column: change_830_to_9
adj_R2: -0.000, P-values: (0.39806577867145676, 0.38506986375387753), column: change_8_to_9
adj_R2: 0.004, P-values: (0.03801048248930684, 0.005871351692641564), column: distance
adj_R2: 0.005, P-values: (0.007611843001945874, 0.00351191034499684), column: num_stations_1mi
adj_R2: -0.001, P-values: (0.9685077762487294, 0.9501893086905262), column: num_stations_2mi
adj_R2: 0.006, P-values: (0.0009687781829920468, 0.0009086458466180611), column: all_stations


In [11]:
remaining_var = df2.drop(['change_8_to_9', 'change_830_to_9','change_8_to_830'], axis=1)
remaining_var.head()

,slots,avail_bikes_9am,distance,num_stations_1mi,num_stations_2mi,all_stations,transit_cat
0,19,13,653,8,2,10,3
1,15,10,384,9,1,10,3
2,15,13,432,10,0,10,3
3,15,7,500,10,0,10,3
4,39,12,1421,5,5,10,3


In [12]:
included_df = df2[['change_830_to_9','change_8_to_830']]
included_df

,change_830_to_9,change_8_to_830
0,-1,-1
1,-1,-1
2,-1,0
3,-2,-2
4,0,0
...,...,...
1658,0,0
1659,0,0
1660,-1,0
1661,0,0


In [13]:
X = [sm.add_constant(pd.merge(included_df,remaining_var[column], right_index = True, left_index = True)) for column in remaining_var.columns] 
X[2]

,const,change_830_to_9,change_8_to_830,distance
0,1.0,-1,-1,653
1,1.0,-1,-1,384
2,1.0,-1,0,432
3,1.0,-2,-2,500
4,1.0,0,0,1421
...,...,...,...,...
1658,1.0,0,0,3007
1659,1.0,0,0,3133
1660,1.0,-1,0,1193
1661,1.0,0,0,1838


In [14]:
Models = [sm.OLS(y,x) for x in X] #list of models
Results = [model.fit() for model in Models] #list of results
Adj_Rsquared = [results.rsquared_adj for results in Results] #list of rsquared
Pval = [results.pvalues for results in Results] #list of list of p-values

for i in range(len(Adj_Rsquared)):
     print(f'adj_R2: {Adj_Rsquared[i]:.3f}, P-values: {*Pval[i],}, column: {remaining_var.columns[i]}')

adj_R2: 1.000, P-values: (2.2433488881410967e-148, 0.0, 0.0, 0.0013371554460581087), column: slots
adj_R2: 1.000, P-values: (3.9212149784721485e-228, 0.0, 0.0, 4.939144575066503e-172), column: avail_bikes_9am
adj_R2: 1.000, P-values: (3.7753071068573634e-34, 0.0, 0.0, 2.5617507207173706e-26), column: distance
adj_R2: 1.000, P-values: (3.4752722097722666e-58, 0.0, 0.0, 1.3943399645890594e-177), column: num_stations_1mi
adj_R2: 1.000, P-values: (2.503491827993701e-84, 0.0, 0.0, 0.0015208516671614057), column: num_stations_2mi
adj_R2: 1.000, P-values: (9.263131177119168e-12, 0.0, 0.0, 0.0041533608960754404), column: all_stations
adj_R2: 1.000, P-values: (6.006122534318659e-05, 0.0, 0.0, 1.9996942001809218e-08), column: transit_cat


In [15]:
#Well! This is a strange issue! R values are all one
#Of course it makes sense for the change between 8-8:30 and 8:30 to 9 to result in some great change, but this suggests
#something very strange could be going on

In [16]:
# Let's run this again but without those two columns included from the beginning...

In [17]:
#now let's create a dataframe just holding the information we care about
df3 = df[['slots','avail_bikes_9am','change_8_to_9','distance','num_stations_1mi','num_stations_2mi','all_stations','transit_cat']]

In [18]:
y = df3['change_8_to_9']
df_new = df3.drop(columns=['change_8_to_9'])

print(df_new.columns)

X = [sm.add_constant(df_new[column]) for column in df_new.columns]
X[1]

Index(['slots', 'avail_bikes_9am', 'distance', 'num_stations_1mi',
       'num_stations_2mi', 'all_stations', 'transit_cat'],
      dtype='object')


,const,avail_bikes_9am
0,1.0,13
1,1.0,10
2,1.0,13
3,1.0,7
4,1.0,12
...,...,...
1658,1.0,3
1659,1.0,0
1660,1.0,7
1661,1.0,3


In [19]:
Models = [sm.OLS(y,x) for x in X] #list of models
Results = [model.fit() for model in Models] #list of results
Adj_Rsquared = [results.rsquared_adj for results in Results] #list of rsquared
Pval = [results.pvalues for results in Results] #list of p-values

In [20]:
for i in range(len(Adj_Rsquared)):
     print(f'adj_R2: {Adj_Rsquared[i]:.3f}, P-values: {*Pval[i],}, column: {df3.columns[i]}')

adj_R2: 0.013, P-values: (0.0007898493290166014, 1.4956518539117485e-06), column: slots
adj_R2: 0.041, P-values: (8.747339303813141e-07, 4.0847238829841874e-17), column: avail_bikes_9am
adj_R2: -0.000, P-values: (0.39806577867145676, 0.38506986375387753), column: change_8_to_9
adj_R2: 0.004, P-values: (0.03801048248930684, 0.005871351692641564), column: distance
adj_R2: 0.005, P-values: (0.007611843001945874, 0.00351191034499684), column: num_stations_1mi
adj_R2: -0.001, P-values: (0.9685077762487294, 0.9501893086905262), column: num_stations_2mi
adj_R2: 0.006, P-values: (0.0009687781829920468, 0.0009086458466180611), column: all_stations


In [21]:
#here we have very low adj. Rs, suggesting these values may not be doing very much
#distance now has a very high p-value

In [22]:
#this can be seen more clearly by running the full model
y = df2['change_8_to_9']
X = df2.drop('change_8_to_9', axis=1)
X = sm.add_constant(X) #adds a column of 1's so the model will contain an intercept

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          change_8_to_9   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 6.675e+31
Date:                Tue, 21 Nov 2023   Prob (F-statistic):               0.00
Time:                        11:35:17   Log-Likelihood:                 53932.
No. Observations:                1663   AIC:                        -1.078e+05
Df Residuals:                    1654   BIC:                        -1.078e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -5.808e-15   6.01e-16  

In [23]:
#Awful

We should investigate what meta-issues we think might be effecting this.

First, the granularity of our sampling may be off. Pulling data every half-hour could be too large of a sampling window.

Second, we are currently unable to track if a bike has been returned and then checked out, so total volume is missing from this data set.

Third, there might not actually be any statistically significant relationship at all.

Continuing and developing this project will require several additional step.